# Import Libraries

In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
from statistics import mode
from collections import Counter, defaultdict

# Compiled Metrics Results

In [57]:
# calculate RMSE
modes = ['LSTM' ,'GRU','RNN']
EMITEN = 'TLKM'
TARGET_DAYS = [1, 5, 10, 20, 50]
MODEL_LIST = ['Price Prediction', 'Deep Learning', 'Machine Learning']

ensemble_results_dir = f'Ensemble Result/{EMITEN}'
if not os.path.exists(ensemble_results_dir):
    os.makedirs(ensemble_results_dir)

result_file_path = f'{ensemble_results_dir}/{EMITEN}_Results.csv'
header_results = f'emiten,model_type,target_day,accuracy,precision,recall,f1_score'
with open(result_file_path, 'w') as writer:
    writer.write(f'{header_results}\n')

all_predictions = []
model_accuracies = {}

for MODEL in MODEL_LIST:
    for TARGET_DAY in TARGET_DAYS:
        if MODEL == 'Price Prediction':
            data_path = f'Result {MODEL}/{EMITEN}/LSTM_{EMITEN}_Target_{TARGET_DAY}.csv'
        else:
            data_path = f'Result {MODEL}/{EMITEN}/{EMITEN}_Target_{TARGET_DAY}.csv'
    
        result_df = pd.read_csv(data_path)
        test_df = result_df[result_df['type'] == 'test']
        testCategoryPredict = test_df['prediction'].values
        testCatY = test_df['ground_truth'].values

        test_acc = round(accuracy_score(list(testCatY), testCategoryPredict), 4)
        test_prec = round(precision_score(list(testCatY), testCategoryPredict, average='weighted', zero_division=0), 4)
        test_rec = round(recall_score(list(testCatY), testCategoryPredict, average='weighted', zero_division=0), 4)
        test_f1 = round(f1_score(list(testCatY), testCategoryPredict, average='weighted', zero_division=0), 4)

        test_results = f'{EMITEN},{MODEL},{TARGET_DAY},{test_acc},{test_prec},{test_rec},{test_f1}'

        with open(result_file_path, 'a') as writer:
            writer.write('{}\n'.format(test_results))
        all_predictions.append(testCategoryPredict)
        model_accuracies[(MODEL, TARGET_DAY)] = test_acc
        print(f'{MODEL} Target {TARGET_DAY} Done')

Price Prediction Target 1 Done
Price Prediction Target 5 Done
Price Prediction Target 10 Done
Price Prediction Target 20 Done
Price Prediction Target 50 Done
Deep Learning Target 1 Done
Deep Learning Target 5 Done
Deep Learning Target 10 Done
Deep Learning Target 20 Done
Deep Learning Target 50 Done
Machine Learning Target 1 Done
Machine Learning Target 5 Done
Machine Learning Target 10 Done
Machine Learning Target 20 Done
Machine Learning Target 50 Done


### Ensemble Model

In [58]:
total_ensemble = len(TARGET_DAYS)
ensemble_preds = np.zeros([all_predictions[0].shape[0], total_ensemble])
ensemble_preds.shape

np_all_prediction = all_predictions[0]
np_all_prediction = np_all_prediction.reshape([-1, 1])

## concat all prediction
for preds in all_predictions[1:]:
    np_all_prediction = np.hstack((np_all_prediction, preds.reshape([-1, 1])))

# concat prediction for same targe dat
for i in range(total_ensemble):
    for j in range(np_all_prediction.shape[0]):
        target = 'None'
        tmp = np_all_prediction[j]
        
        # find prediction for the same target day
        tmp = (tmp[i + (total_ensemble * 0)], tmp[i + (total_ensemble * 1)], tmp[i + total_ensemble * 2])
        label = Counter(tmp).most_common()[0]
        if label[1] == 1:
            label = 1
        else:
            label = label[0]
        ensemble_preds[j, i] = label

In [59]:
result_df = pd.read_csv(data_path)
for i in range(total_ensemble):
    result_df = pd.read_csv(data_path)
    test_df = result_df[result_df['type'] == 'test']
    testCatY = test_df['ground_truth'].values
    testCategoryPredict = ensemble_preds[:, i]

    test_acc = round(accuracy_score(list(testCatY), testCategoryPredict), 4)
    test_prec = round(precision_score(list(testCatY), testCategoryPredict, average='weighted', zero_division=0), 4)
    test_rec = round(recall_score(list(testCatY), testCategoryPredict, average='weighted', zero_division=0), 4)
    test_f1 = round(f1_score(list(testCatY), testCategoryPredict, average='weighted', zero_division=0), 4)

    test_results = f'{EMITEN},Ensemble Model,{TARGET_DAYS[i]},{test_acc},{test_prec},{test_rec},{test_f1}'

    with open(result_file_path, 'a') as writer:
        writer.write('{}\n'.format(test_results))

### Weighted Ensemble Model

In [60]:
weighted_ensemble_preds = np.zeros([all_predictions[0].shape[0], total_ensemble])
weighted_ensemble_preds.shape

# Weighted ensemble prediction based on weighted sum
for i in range(total_ensemble):
    for j in range(np_all_prediction.shape[0]):
        class_weight_sum = defaultdict(float) 

        for k, MODEL in enumerate(MODEL_LIST):
            pred_class = np_all_prediction[j, i + (total_ensemble * k)]
            weight = model_accuracies[(MODEL, TARGET_DAYS[i])]
            class_weight_sum[pred_class] += weight

        final_prediction = max(class_weight_sum, key=class_weight_sum.get)
        weighted_ensemble_preds[j, i] = final_prediction
        
result_df = pd.read_csv(data_path)
for i in range(total_ensemble):
    result_df = pd.read_csv(data_path)
    test_df = result_df[result_df['type'] == 'test']
    testCatY = test_df['ground_truth'].values
    testCategoryPredict = weighted_ensemble_preds[:, i]

    test_acc = round(accuracy_score(list(testCatY), testCategoryPredict), 4)
    test_prec = round(precision_score(list(testCatY), testCategoryPredict, average='weighted', zero_division=0), 4)
    test_rec = round(recall_score(list(testCatY), testCategoryPredict, average='weighted', zero_division=0), 4)
    test_f1 = round(f1_score(list(testCatY), testCategoryPredict, average='weighted', zero_division=0), 4)

    test_results = f'{EMITEN},Weighted Ensemble,{TARGET_DAYS[i]},{test_acc},{test_prec},{test_rec},{test_f1}'

    with open(result_file_path, 'a') as writer:
        writer.write('{}\n'.format(test_results))

In [61]:
# Load the results into a dataframe to check the final results
tmp = pd.read_csv(result_file_path)
tmp

,emiten,model_type,target_day,accuracy,precision,recall,f1_score
0,TLKM,Price Prediction,1,0.6352,0.4138,0.6352,0.5012
1,TLKM,Price Prediction,5,0.6352,0.4686,0.6352,0.5289
2,TLKM,Price Prediction,10,0.8238,0.7434,0.8238,0.7799
3,TLKM,Price Prediction,20,0.9016,0.8595,0.9016,0.8679
4,TLKM,Price Prediction,50,0.9098,0.9115,0.9098,0.9034
5,TLKM,Deep Learning,1,0.6107,0.4594,0.6107,0.5052
6,TLKM,Deep Learning,5,0.6598,0.6398,0.6598,0.6359
7,TLKM,Deep Learning,10,0.9262,0.9234,0.9262,0.9243
8,TLKM,Deep Learning,20,0.9344,0.9395,0.9344,0.9360
9,TLKM,Deep Learning,50,0.9016,0.9145,0.9016,0.9052
